# Data Exploration

## Import Packages

In [43]:
import pandas as pd
import seaborn as sns
import sqlite3

## Box Office Mojo Data

In [2]:
movies_gross_df = pd.read_csv('./data/bom.movie_gross.csv.gz')
movies_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


## IMDB Data

In [3]:
conn = sqlite3.connect("./data/im.db")

In [4]:
imdb_movie_info = pd.read_sql("""SELECT * FROM movie_basics;""", conn)

### Top Movies by # Votes

In [10]:
top_movies_by_votes = pd.read_sql("""SELECT genres, primary_title, averagerating, start_year, numvotes
                                FROM movie_basics
                                JOIN movie_ratings
                                    USING(movie_id)
                                ORDER BY numvotes DESC
;""", conn)

In [11]:
top_movies_by_votes

,genres,primary_title,averagerating,start_year,numvotes
0,"Action,Adventure,Sci-Fi",Inception,8.8,2010,1841066
1,"Action,Thriller",The Dark Knight Rises,8.4,2012,1387769
2,"Adventure,Drama,Sci-Fi",Interstellar,8.6,2014,1299334
3,"Drama,Western",Django Unchained,8.4,2012,1211405
4,"Action,Adventure,Sci-Fi",The Avengers,8.1,2012,1183655
...,...,...,...,...,...
73851,Comedy,Columbus,5.8,2018,5
73852,"Comedy,Horror",BADMEN with a good behavior,9.2,2018,5
73853,Romance,July Kaatril,9.0,2019,5
73854,Documentary,Swarm Season,6.2,2019,5


### Top Genres by Count

In [19]:
top_genres_by_count = pd.read_sql("""SELECT genres, COUNT(*) as amount
                                        FROM movie_basics
                                        GROUP BY GENRES
                                        ORDER BY amount DESC
;""", conn)

In [20]:
top_genres_by_count

,genres,amount
0,Documentary,32185
1,Drama,21486
2,Comedy,9177
3,None,5408
4,Horror,4372
...,...,...
1081,"Action,Animation,Music",1
1082,"Action,Animation,History",1
1083,"Action,Animation,Documentary",1
1084,"Action,Animation,Biography",1


### Top Genres by Rating

In [50]:
top_genres_by_rating = pd.read_sql("""SELECT genres, AVG(averagerating) as average_rating
                                        FROM movie_basics
                                        JOIN movie_ratings
                                            USING(movie_id)
                                        GROUP BY GENRES
                                        ORDER BY average_rating DESC
;""", conn)

In [51]:
top_genres_by_rating

,genres,average_rating
0,"Comedy,Documentary,Fantasy",9.4
1,"Documentary,Family,Musical",9.3
2,"History,Sport",9.2
3,"Music,Mystery",9.0
4,Game-Show,9.0
...,...,...
919,"Crime,Music",2.4
920,"History,Sci-Fi,Thriller",2.3
921,"Adventure,Crime,Romance",2.3
922,"Adult,Horror",2.0


In [7]:
#conn.close()